# MDG NAP Preliminary Analysis

In [108]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from scipy import stats

init_notebook_mode(connected=True)

engine = create_engine('postgresql://postgres@localhost:5433/cartodb_user_fddc7a0f-c829-4ec0-a4b8-212ec7ce9ad1_db')
sql_rast_mean_by_region = """
SELECT {group_by},
(ST_SummaryStatsAgg(ST_Clip(the_raster_webmercator, the_geom_webmercator, {na_value}, true), 1, true)).* 
FROM {raster}, {poly}
WHERE ST_Intersects(the_raster_webmercator, the_geom_webmercator)
GROUP BY {group_by};
"""

# General function to compare series
def plot_comparison(x, y, labels, x_label, y_label, size=None, trend=True):
    if size is None:
        trace = go.Scatter(
            x = x,
            y = y,
            text = labels,
            mode = 'markers',
        )
    else:
        trace = go.Scatter(
            x = x,
            y = y,
            text = labels,
            mode = 'markers',
            marker = dict (
                sizemode = 'area',
                size = size
            )
        )
    if trend:
        slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
        trendline = slope * x + intercept
        r, p = stats.pearsonr(x, y)
        trend_trace = go.Scatter(
                  x=x, 
                  y=trendline, 
                  mode='lines',
                  opacity = 0.5,
                  hoverinfo='none'
        )
        annotation = go.Annotation(
                  x=.9,
                  y=.9,
                  xref = 'paper',
                  yref = 'paper',
                  text = 'r = {:.2}, p = {:.2}'.format(r, p),
                  showarrow=False,
                  font=go.Font(size=16, family='Arial, italic')
        )
        layout = go.Layout(
            xaxis=dict(title=x_label),
            yaxis=dict(title=y_label),
            annotations = [annotation],
            showlegend = False
        )
        data = [trace, trend_trace]
    else:
        layout = go.Layout(
            xaxis=dict(title=x_label),
            yaxis=dict(title=y_label)
        )
        data = [trace]
    fig = go.Figure(data=data, layout=layout)
    return(iplot(fig, filename='basic-scatter'))

# General function to compare two rasters by adm2 region
def get_rast_mean_by_region(rast, region="(select * from gadm28_adm2 where iso in ('MDG')) as regions"):
    key = {'raster': rast[1], # This is the raster table name
           'poly': region,
           'na_value': -9999,
           'group_by': 'name_2'}
    df = pd.read_sql_query(sql_rast_mean_by_region.format(**key), con=engine)
    df['variable'] = rast[0] # This is a short name for the raster
    return(df)

## Agriculture / water

### Climate trends

In [25]:
rast_comp =({'precip':'total_annual_rainfall_multimodel_pctdiff_mean_85_50_mdg',
             'temp':'total_annual_temperature_multimodel_pctdiff_mean_8570_mdg'})
df = pd.concat([get_rast_mean_by_region(rast) for rast in rast_comp.items()])

In [109]:
dfpivot = df.pivot(index='name_2', columns='variable', values='mean')
plot_comparison(dfpivot['temp'], dfpivot['precip'], dfpivot.index,
                x_label='Temperature change (°C)', y_label='Precipitation change (%)')

### Health versus climate

In [4]:
# Pull precipitation trends
key = {'raster': 'total_annual_rainfall_multimodel_pctdiff_mean_85_50_mdg',
       'poly': 'sdr_subnational_data_dhs_2008',
       'na_value': -9999,
       'group_by': 'dhsregen'}
df_rain = pd.read_sql_query(sql_rast_mean_by_region.format(**key), con=engine)
df_rain = df_rain.rename(columns={'mean':'precip'})

# Pull temperature trends
key = {'raster': 'total_annual_temperature_multimodel_pctdiff_mean_8570_mdg',
       'poly': 'sdr_subnational_data_dhs_2008',
       'na_value': -9999,
       'group_by': 'dhsregen'}
df_temp = pd.read_sql_query(sql_rast_mean_by_region.format(**key), con=engine)
df_temp = df_temp.rename(columns={'mean':'temp'})

# Pull population
key = {'raster': 'lspop2014',
       'poly': 'sdr_subnational_data_dhs_2008',
       'na_value': -9999,
       'group_by': 'dhsregen'}
df_pop = pd.read_sql_query(sql_rast_mean_by_region.format(**key), con=engine)
df_pop = df_pop.rename(columns={'sum':'pop'})

# Merge both climate datasets
df_clim = pd.merge(df_rain[['dhsregen', 'precip']], df_temp[['dhsregen', 'temp']])
df_clim = pd.merge(df_clim, df_pop[['dhsregen', 'pop']])

# Pull health data
df_health = pd.read_sql_query('select cnnutscha2 as stunting, cmecmrcimr as imr, mlitnapacc as pct_access_nets, dhsregen from sdr_subnational_data_dhs_2008', con=engine)
df_health = pd.merge(df_clim, df_health)
df_health = df_health.set_index('dhsregen')
df_health['size'] = (df_health['pop'] / df_health['pop'].max()) * 1000

#### Stunting

In [110]:
plot_comparison(df_health['stunting'], df_health['precip'], df_health.index,
                x_label='Children that are stunted (%)', y_label='Precipitation change (%)',
                size=df_health['size'], trend=True)

In [99]:
plot_comparison(df_health['stunting'], df_health['temp'], df_health.index,
                x_label='Children that are stunted (%)', y_label='Temperature change (%)',
                size=df_health['size'])

#### Infant mortality rates

In [100]:
plot_comparison(df_health['imr'], df_health['precip'], df_health.index,
                x_label='Infant mortality rate', y_label='Precipitation change (%)',
                size=df_health['size'])

In [101]:
plot_comparison(df_health['imr'], df_health['temp'], df_health.index,
                x_label='Infant mortality rate', y_label='Temperature change (%)',
                size=df_health['size'])

#### Access to bed nets

In [102]:
plot_comparison(df_health['pct_access_nets'], df_health['precip'], df_health.index,
                x_label='Percentage with access to insecticide treated nets', y_label='Precipitation change (%)',
                size=df_health['size'])

In [103]:
plot_comparison(df_health['pct_access_nets'], df_health['temp'], df_health.index,
                x_label='Percentage with access to insecticide treated nets', y_label='Temperature change (%)',
                size=df_health['size'])

#### Climate and food security

In [11]:
# Pull precipitation trends
key = {'raster': 'total_annual_rainfall_multimodel_pctdiff_mean_85_50_mdg',
       'poly': 'communes_poverty_fid_2007',
       'na_value': -9999,
       'group_by': 'first_nom_'}
df_rain = pd.read_sql_query(sql_rast_mean_by_region.format(**key), con=engine)
df_rain = df_rain.rename(columns={'mean':'precip'})

# Pull temperature trends
key = {'raster': 'total_annual_temperature_multimodel_pctdiff_mean_8570_mdg',
       'poly': 'communes_poverty_fid_2007',
       'na_value': -9999,
       'group_by': 'first_nom_'}
df_temp = pd.read_sql_query(sql_rast_mean_by_region.format(**key), con=engine)
df_temp = df_temp.rename(columns={'mean':'temp'})

# Pull population
key = {'raster': 'lspop2014',
       'poly': 'communes_poverty_fid_2007',
       'na_value': -9999,
       'group_by': 'first_nom_'}
df_pop = pd.read_sql_query(sql_rast_mean_by_region.format(**key), con=engine)
df_pop = df_pop.rename(columns={'sum':'pop'})
#print df_pop[df_pop['pop'] == df_pop['pop'].max()]
#print df_pop['pop'].sum()

# Merge climate and population datasets
df_clim = pd.merge(df_rain[['first_nom_', 'precip']], df_temp[['first_nom_', 'temp']])
df_clim = pd.merge(df_clim, df_pop[['first_nom_', 'pop']])

# Pull health data
df_commune = pd.read_sql_query('select poor_per as food_insecurity, first_nom_ from communes_poverty_fid_2007', con=engine)
df_commune = pd.merge(df_clim, df_commune)
df_commune = df_commune.set_index('first_nom_')
df_commune['size'] = (df_commune['pop'] / df_commune['pop'].max()) * 1000

In [104]:
plot_comparison(df_commune['food_insecurity'], df_commune['precip'], df_commune.index,
                x_label='Percentage food insecure in ??', y_label='Precipitation change (%)',
                size=df_commune['size'])

In [105]:
plot_comparison(df_commune['food_insecurity'], df_commune['temp'], df_commune.index,
                x_label='Percentage food insecure in ??', y_label='Temperature change (%)',
                size=df_commune['size'])

#### Food security versus health stress

#### Climate versus essential natural capital for food security

#### Climate versus rice production

## Disaster risk reduction

#### Literacy/governance/institutions and coastal vulnerability

#### Coral and mangroves versus coastal vulnerability